## Imports and Defs

In [ ]:
import assets.helper as b3
import assets.functions as run



## Content

#### Merge Mix and Match all

In [ ]:
try:
    math = run.load_pkl(f'{b3.app_folder}math')
except Exception as e:
    math = run.get_math()
    math = run.save_pkl(math, f'{b3.app_folder}math')


In [ ]:

try:
    setorial = run.load_pkl(f'{b3.app_folder}setorial')
except Exception as e:
    setorial = run.get_classificacao_setorial(setorial='')
    setorial = run.save_pkl(setorial, f'{b3.app_folder}setorial')
# setorial.head(3)

In [318]:

try:
    b3_cvm = run.load_pkl(f'{b3.app_folder}b3_cvm')
except Exception as e:
    b3_cvm = run.b3_grab(b3.search_url)
    b3_cvm = run.save_pkl(b3_cvm, f'{b3.app_folder}b3_cvm')


KeyboardInterrupt: 

In [ ]:
# SETOR: O setor econômico mais amplo ao qual a empresa pertence.
# SUBSETOR: Uma categorização mais específica dentro do setor mais amplo.
# SEGMENTO: Uma classificação ainda mais granular do negócio da empresa.
# DENOM_CIA: Esta é a denominação ou nome da empresa.
# COMPANHIA: Nome ou denominação oficial da empresa listada.
# PREGAO: Refere-se ao nome pelo qual a empresa é conhecida no pregão da bolsa de valores.
# LISTAGEM: Categoria ou segmento de listagem da empresa na bolsa de valores, que pode indicar o nível de governança corporativa ou outros critérios.
# TICK: Abreviação ou símbolo da empresa usada no mercado de ações.
# TICKERS: Símbolos de negociação da empresa em diferentes mercados ou plataformas.
# CD_CVM: Este poderia ser um código ou identificador único relacionado à empresa, possivelmente relacionado à Comissão de Valores Mobiliários do Brasil (CVM).
# CVM: Código ou identificador relacionado à empresa na Comissão de Valores Mobiliários, o órgão regulador do mercado de capitais no Brasil.
# ISIN: Número de Identificação Internacional de Valores Mobiliários – um identificador único para valores mobiliários.
# CNPJ_CIA: Este é o número do Cadastro Nacional da Pessoa Jurídica (CNPJ) da empresa, um identificador único para empresas no Brasil.
# CNPJ: Cadastro Nacional da Pessoa Jurídica – é o número de identificação das empresas brasileiras.
# SITE: Site oficial ou página relevante da empresa.
# ATIVIDADE: Descreve a principal atividade de negócios da empresa.

# ANO: Este é o ano ao qual os dados se referem.
# DT_REFER: Esta é a data de referência para a entrada de dados.
# DT_FIM_EXERC: Esta é a data final para o exercício ou período de relato financeiro.
# DT_INI_EXERC: Esta poderia ser a data inicial para o exercício ou período de relato financeiro.

# AGRUPAMENTO: Isso descreve o nível de agregação dos dados. Por exemplo, 'con' pode indicar dados consolidados.
# BALANCE_SHEET: Isso indica a seção específica da demonstração financeira, como Balanço Patrimonial ('BPA').
# GRUPO_DFP: Isso representa o tipo de grupo de demonstração financeira. Por exemplo, 'DF Consolidado - Balanço Patrimonial Ativo' sugere que é um balanço patrimonial consolidado focado em ativos.
# CD_CONTA: Este poderia ser um código ou identificador único relacionado a uma conta específica ou item de linha na demonstração financeira.
# DS_CONTA: Descreve a conta específica ou item de linha na demonstração financeira, como 'Ativo Total'.

# VL_CONTA: Representa o valor associado à conta específica ou item de linha.
# MOEDA: Isso indica a moeda na qual os valores são representados. 'REAL' sugere Real Brasileiro.
# ESCALA_MOEDA: Isso fornece a escala ou unidade para os valores monetários. 'MIL' pode indicar que os valores estão em milhares.

# ST_CONTA_FIXA: Pode indicar o status ou tipo de conta. O significado de valores como 'S' dependeria do contexto dos dados.
# COLUNA_DF: O propósito desta coluna não é imediatamente claro a partir da amostra. Pode representar algum tipo de classificação ou categorização relacionada aos dados financeiros.

# ESCRITURADOR: Entidade ou empresa responsável por registrar ou gerenciar os valores mobiliários da empresa.
# ACIONISTAS: Informações ou identificadores relacionados aos acionistas da empresa.

# FILENAME: Este é o arquivo de onde os dados são originados. Ele fornece o nome do arquivo que contém a respectiva entrada de dados.
# DEMONSTRATIVO: Este representa o tipo de demonstração financeira. Pode indicar se os dados são de um relatório intermediário (como 'itr') ou de outro tipo de relatório financeiro.
# VERSAO: Isso pode representar a versão ou iteração dos dados/relatórios financeiros.

columns = [
    'SETOR_x',
    'SUBSETOR_x',
    'SEGMENTO_x',
    'DENOM_CIA',
        # 'COMPANHIA',
    'PREGAO',
    'LISTAGEM',
    'TICK',
    'TICKERS',
    'CD_CVM',
        # 'CVM',
        # 'ISIN',
    'CNPJ_CIA',
        # 'CNPJ',
    'SITE',
    'ATIVIDADE',
        # 'ANO',
    'DT_REFER',
        # 'DT_FIM_EXERC',
        # 'DT_INI_EXERC',
    'AGRUPAMENTO',
    'BALANCE_SHEET',
    # 'GRUPO_DFP',
    'CD_CONTA',
    'DS_CONTA',
    'VL_CONTA',
    # 'MOEDA',
    # 'ESCALA_MOEDA',
    # 'ST_CONTA_FIXA',
    # 'COLUNA_DF',
    'ESCRITURADOR',
    'ACIONISTAS', 
    # 'FILENAME', 
    # 'DEMONSTRATIVO', 
    # 'VERSAO',
]


In [ ]:
b3_cvm.keys()

In [ ]:
columns = ['FILENAME', 'DEMONSTRATIVO', 'BALANCE_SHEET', 'ANO', 'AGRUPAMENTO',
       'CNPJ_CIA', 'DT_REFER', 'VERSAO', 'DENOM_CIA', 'CD_CVM', 'GRUPO_DFP',
       'MOEDA', 'ESCALA_MOEDA', 'DT_FIM_EXERC', 'CD_CONTA', 'DS_CONTA',
       'VL_CONTA', 'ST_CONTA_FIXA', 'DT_INI_EXERC', 'COLUNA_DF', 'COMPANHIA',
       'PREGAO', 'TICK', 'LISTAGEM', 'TICKERS', 'ISIN', 
       'ATIVIDADE', 'SETOR', 'SUBSETOR', 'SEGMENTO', 'SITE', 'ESCRITURADOR',]
df = b3_cvm['CONSUMO CICLICO'][columns].set_index('DT_REFER')
df = convert_columns(df)
df['DENOM_CIA'].unique()

#### Intel


In [1]:
import pandas as pd

In [ ]:
b3_cvm = run.load_pkl(f'{b3.app_folder}b3_cvm')


In [2]:
df = pd.read_csv('BENS INDUSTRIAIS_df.csv')
# df = pd.read_csv('COMUNICACOES_df.csv')


In [3]:
def filter_dataframe(df, rules):
    df_new = pd.DataFrame()

    for line, conditions in rules:
        mask = pd.Series([True] * len(df))
        line_CD, line_DS = line.split(' - ', 1)
       
        print()  # Debug print
        for condition, criteria in conditions:
            condition_part = condition.split('_')[1] if '_' in condition else condition
            
            if condition_part in ["startswith", "startswith_not", "endswith", "endswith_not", "contains", "contains_not"]:
                if type(criteria) is str:
                    criteria = [criteria]
                if type(criteria) is tuple:
                    criteria = list(criteria)
                if condition_part in ["contains", "startswith", "endswith"]:
                    criteria = [word.lower() for word in criteria]
                    criteria_str = '|'.join(criteria)  # Convert to string

            # CD_CONTA criteria
            if condition == 'conta_exact':
                mask &= df['CD_CONTA'] == criteria
                print(line, condition, criteria, mask.sum())  # Debug print
            elif condition == 'conta_startswith':
                mask &= df['CD_CONTA'].str.startswith(criteria_str)
                print(line, condition, criteria_str, mask.sum())  # Debug print
            elif condition == 'conta_level_min':
                mask &= df['CD_CONTA'].str.count('\.') + 1 >= criteria
                print(line, condition, criteria, mask.sum())  # Debug print
            elif condition == 'conta_level_max':
                mask &= df['CD_CONTA'].str.count('\.') + 1 <= criteria
            elif condition == 'conta_level_exact':
                mask &= df['CD_CONTA'].str.count('\.') + 1 == criteria
                print(line, condition, criteria, mask.sum())  # Debug print
                print(line, condition, criteria, mask.sum())  # Debug print
            elif condition == 'conta_contains_not':
                mask &= ~df['CD_CONTA'].isin(criteria)
                print(line, condition, criteria, mask.sum())  # Debug print
            
            # DS_CONTA criteria
            elif condition == 'descricao_contains':
                mask &= df['DS_CONTA'].str.lower().str.contains(criteria_str)
                print(line, condition, criteria_str, mask.sum())  # Debug print
            elif condition == 'descricao_contains_not':
                mask &= ~df['DS_CONTA'].str.lower().str.contains(criteria_str)
                print(line, condition, criteria_str, mask.sum())  # Debug print

        matching_rows =  df[mask].copy()
        matching_rows['CD_CONTA'] = line_CD
        matching_rows['DS_CONTA'] = line_DS
        matching_rows['CD_CONTA_original'] =  df[mask]['CD_CONTA']
        matching_rows['DS_CONTA_original'] =  df[mask]['DS_CONTA']
        df_new = pd.concat([df_new, matching_rows])
        
    return df_new



In [ ]:
from assets.rules import rules


In [ ]:
# Now, to check if these descriptions appear in other '2.02.X' subsets
other_subsets_mask = df['CD_CONTA'].str.startswith(subset.rsplit('.', 1)[0] + '.') & ~df['CD_CONTA'].str.startswith(subset)
other_subsets_df = df[other_subsets_mask]

# Filter other_subsets_df to only have rows with the same descriptions as in our original subset
conflicting_descriptions_df = other_subsets_df[other_subsets_df['DS_CONTA'].isin(subset_df['DS_CONTA'].value_counts().index)]

# Group by CD_CONTA and DS_CONTA to see the distribution
conflicting_entries = conflicting_descriptions_df.groupby(['CD_CONTA', 'DS_CONTA']).size().reset_index(name='count')

# conflicting_entries


In [ ]:
# Grouping by DS_CONTA and aggregating the CD_CONTA values and their counts
aggregated_df = conflicting_entries.groupby('DS_CONTA').agg({
    'CD_CONTA': lambda x: list(x),
    'count': 'sum'
}).reset_index()

# Sorting the dataframe by count in descending order for better clarity
aggregated_df = aggregated_df.sort_values(by='count', ascending=False)
aggregated_df

# Breaking down the aggregated dataframe
expanded_df = aggregated_df.explode('CD_CONTA')

# Merging with the original conflicting_entries dataframe to get the count for each CD_CONTA
expanded_df = expanded_df.merge(conflicting_entries, on=['CD_CONTA', 'DS_CONTA'], suffixes=('_total', '_individual'))

# Sorting the dataframe by the total count and CD_CONTA for better clarity
expanded_df = expanded_df.sort_values(by=['count_total', 'CD_CONTA'], ascending=[False, True])
expanded_df = expanded_df[['DS_CONTA', 'CD_CONTA', 'count_individual']]
expanded_df

expanded_df_sorted = expanded_df.sort_values(by=['DS_CONTA', 'count_individual'], ascending=[True, True])
# expanded_df_sorted.head(50)


In [ ]:
# Recreating the conflicting_entries dataframe based on the initial code provided

# Creating the mask for the subset and level
m_subset = df['CD_CONTA'].str.startswith(subset)
m_level = df['CD_CONTA'].str.count('\.') == level

# Applying the mask
subset_df = df[m_subset & m_level]

# Now, to check if these descriptions appear in other subsets under the main subset (e.g., '2.02.X')
other_subsets_mask = df['CD_CONTA'].str.startswith(subset.rsplit('.', 1)[0] + '.') & ~df['CD_CONTA'].str.startswith(subset)
other_subsets_df = df[other_subsets_mask]

# Filter other_subsets_df to only have rows with the same descriptions as in our original subset
conflicting_descriptions_df = other_subsets_df[other_subsets_df['DS_CONTA'].isin(subset_df['DS_CONTA'].value_counts().index)]

# Group by CD_CONTA and DS_CONTA to see the distribution
conflicting_entries = conflicting_descriptions_df.groupby(['CD_CONTA', 'DS_CONTA']).size().reset_index(name='count')

# Extracting the CD_CONTA up to the level specified (one level up from the original level)
conflicting_entries['CD_CONTA_up_one_level'] = conflicting_entries['CD_CONTA'].str.rsplit(pat='.', n=1).str[0]

# Grouping by this new level and DS_CONTA
grouped_conflicting_up_one_level = conflicting_entries.groupby(['CD_CONTA_up_one_level', 'DS_CONTA']).agg({'count': 'sum'}).reset_index()

# Sorting the results for clarity
grouped_conflicting_up_one_level = grouped_conflicting_up_one_level.sort_values(by=['DS_CONTA', 'CD_CONTA_up_one_level', 'count'], ascending=[True, True, False])

grouped_conflicting_up_one_level


In [ ]:
subset

In [ ]:
mm = df['CD_CONTA'].str.startswith('3.')
mm &= df['CD_CONTA'].str.count('\.') >= level
mm &= df['CD_CONTA'].str.count('\.') <= level
mm &= df['DS_CONTA'].str.lower().str.contains('descontinuad')
mm &= ~df['DS_CONTA'].str.lower().str.contains('lucro')
df[mm][['CD_CONTA', 'DS_CONTA']].drop_duplicates()

##### Dados Abertos

In [ ]:
def read(file):
    path = "C:\\Users\\faust\\OneDrive\\Área de Trabalho\\dados abertos\\"
    df = pd.read_csv(path+file+".csv", sep=';', encoding='latin1')
    return df.head(25)



In [ ]:
file = "fca_cia_aberta_2010"
fca_cia_aberta_2010 = read(file)
# link para o NSD do formulário cadastral
fca_cia_aberta_2010

In [ ]:
file = "fca_cia_aberta_auditor_2010"
fca_cia_aberta_auditor_2010 = read(file)
# informações dos auditores CNPJ e CPF, datas dos auditores CPF
fca_cia_aberta_auditor_2010

In [ ]:
file = "fca_cia_aberta_canal_divulgacao_2010"
fca_cia_aberta_canal_divulgacao_2010 = read(file)
# Onde as DRE são divulgadas
fca_cia_aberta_canal_divulgacao_2010

In [ ]:
file = "fca_cia_aberta_departamento_acionistas_2010"
fca_cia_aberta_departamento_acionistas_2010 = read(file)
# Endereços dos DRI
fca_cia_aberta_departamento_acionistas_2010

In [ ]:
file = "fca_cia_aberta_dri_2010"
fca_cia_aberta_dri_2010 = read(file)
# NOMES e endereços dos DRI
fca_cia_aberta_dri_2010

In [ ]:
file = "fca_cia_aberta_endereco_2010"
fca_cia_aberta_endereco_2010 = read(file)
# Endereço completo do DRI
fca_cia_aberta_endereco_2010

In [ ]:
file = "fca_cia_aberta_geral_2010"
fca_cia_aberta_geral_2010 = read(file)
# Cadastro CVM, Atividade, Descrição e Controle Acionário
fca_cia_aberta_geral_2010

In [ ]:
file = "fca_cia_aberta_pais_estrangeiro_negociacao_2010"
fca_cia_aberta_pais_estrangeiro_negociacao_2010 = read(file)
# País estrangeiro... ?
fca_cia_aberta_pais_estrangeiro_negociacao_2010

In [ ]:
file = "fca_cia_aberta_valor_mobiliario_2010"
fca_cia_aberta_valor_mobiliario_2010 = read(file)
# Valor mobiliário, Mercado e Segmento
fca_cia_aberta_valor_mobiliario_2010

In [ ]:
file = "fre_cia_aberta_2023"
fre_cia_aberta_2023 = read(file)
# Link do Documento
fre_cia_aberta_2023

In [ ]:
file = "fre_cia_aberta_acao_entregue_2023"
fre_cia_aberta_acao_entregue_2023 = read(file)
# Remuneração da Diretoria
fre_cia_aberta_acao_entregue_2023

In [ ]:
file = "fre_cia_aberta_administrador_declaracao_genero_2023"
fre_cia_aberta_administrador_declaracao_genero_2023 = read(file)
# Gênero dos administradores
fre_cia_aberta_administrador_declaracao_genero_2023

In [ ]:
file = "fre_cia_aberta_administrador_declaracao_raca_2023"
fre_cia_aberta_administrador_declaracao_raca_2023 = read(file)
# Raça dos administradores
fre_cia_aberta_administrador_declaracao_raca_2023

In [ ]:
file = "fre_cia_aberta_administrador_membro_conselho_fiscal_2023"
fre_cia_aberta_administrador_membro_conselho_fiscal_2023 = read(file)
# Membros do Conselho Fiscal
fre_cia_aberta_administrador_membro_conselho_fiscal_2023

In [ ]:
file = "fre_cia_aberta_ativo_imobilizado_2023"
fre_cia_aberta_ativo_imobilizado_2023 = read(file)
# Ativos e Propriedades por empresa
fre_cia_aberta_ativo_imobilizado_2023

In [ ]:
file = "fre_cia_aberta_ativo_intangivel_2023"
fre_cia_aberta_ativo_intangivel_2023 = read(file)
# Ativos e Propriedades por empresa
fre_cia_aberta_ativo_intangivel_2023

In [ ]:
file = "fre_cia_aberta_auditor_2023"
fre_cia_aberta_auditor_2023 = read(file)
# Remuneração por auditor
fre_cia_aberta_auditor_2023

In [ ]:
file = "fre_cia_aberta_auditor_responsavel_2023"
fre_cia_aberta_auditor_responsavel_2023 = read(file)
# Endereço do Auditor
fre_cia_aberta_auditor_responsavel_2023

In [ ]:
file = "fre_cia_aberta_capital_social_2023"
fre_cia_aberta_capital_social_2023 = read(file)
# Modificações no Capital Social e Ações
fre_cia_aberta_capital_social_2023

In [ ]:
file = "fre_cia_aberta_capital_social_aumento_2023"
fre_cia_aberta_capital_social_aumento_2023 = read(file)
# Idem
fre_cia_aberta_capital_social_aumento_2023

In [ ]:
file = "fre_cia_aberta_capital_social_aumento_classe_acao_2023"
fre_cia_aberta_capital_social_aumento_classe_acao_2023 = read(file)
# Em branco
fre_cia_aberta_capital_social_aumento_classe_acao_2023

In [ ]:
file = "fre_cia_aberta_capital_social_classe_acao_2023"
fre_cia_aberta_capital_social_classe_acao_2023 = read(file)
# Preferencial Classe A, B e C
fre_cia_aberta_capital_social_classe_acao_2023

In [ ]:
file = "fre_cia_aberta_capital_social_desdobramento_2023"
fre_cia_aberta_capital_social_desdobramento_2023 = read(file)
# Desdobramentos de Ações
fre_cia_aberta_capital_social_desdobramento_2023

In [ ]:
file = "fre_cia_aberta_capital_social_desdobramento_classe_acao_2023"
fre_cia_aberta_capital_social_desdobramento_classe_acao_2023 = read(file)
# em branco
fre_cia_aberta_capital_social_desdobramento_classe_acao_2023

In [ ]:
file = "fre_cia_aberta_capital_social_reducao_2023"
fre_cia_aberta_capital_social_reducao_2023 = read(file)
# Redução de capital
fre_cia_aberta_capital_social_reducao_2023

In [ ]:
file = "fre_cia_aberta_capital_social_titulo_conversivel_2023"
fre_cia_aberta_capital_social_titulo_conversivel_2023 = read(file)
# Redução de capital
fre_cia_aberta_capital_social_titulo_conversivel_2023

In [ ]:
file = "fre_cia_aberta_direito_acao_2023"
fre_cia_aberta_direito_acao_2023 = read(file)
# ?
fre_cia_aberta_direito_acao_2023

In [ ]:
file = "fre_cia_aberta_historico_emissor_2023"
fre_cia_aberta_historico_emissor_2023 = read(file)
# Constituição do emissor e local
fre_cia_aberta_historico_emissor_2023

In [ ]:
file = "fre_cia_aberta_informacao_financeira_2023"
fre_cia_aberta_informacao_financeira_2023 = read(file)
# DRE Resumido
fre_cia_aberta_informacao_financeira_2023

In [ ]:
file = "fre_cia_aberta_membro_comite_2023"
fre_cia_aberta_membro_comite_2023 = read(file)
# CPF e Remuneração
fre_cia_aberta_membro_comite_2023

In [ ]:
file = "fre_cia_aberta_obrigacao_2023"
fre_cia_aberta_obrigacao_2023 = read(file)
# Obrigações e Dívidas
fre_cia_aberta_obrigacao_2023

In [ ]:
file = "fre_cia_aberta_outro_valor_mobiliario_2023"
fre_cia_aberta_outro_valor_mobiliario_2023 = read(file)
# ?
fre_cia_aberta_outro_valor_mobiliario_2023

In [ ]:
file = "fre_cia_aberta_participacao_sociedade_2023"
fre_cia_aberta_participacao_sociedade_2023 = read(file)
# Participações em outras empresas
fre_cia_aberta_participacao_sociedade_2023

In [ ]:
file = "fre_cia_aberta_participacao_sociedade_valorizacao_acao_2023"
fre_cia_aberta_participacao_sociedade_valorizacao_acao_2023 = read(file)
# ?
fre_cia_aberta_participacao_sociedade_valorizacao_acao_2023

In [ ]:
file = "fre_cia_aberta_posicao_acionaria_2023"
fre_cia_aberta_posicao_acionaria_2023 = read(file)
# Composição acionária por acionista majoritário
fre_cia_aberta_posicao_acionaria_2023[['CNPJ_Companhia', 'Data_Referencia', 'Versao', 'ID_Documento',
       'ID_Acionista', 'Acionista', 'Tipo_Pessoa_Acionista',
       'CPF_CNPJ_Acionista', 'ID_Acionista_Relacionado',
       'Acionista_Relacionado', 'Tipo_Pessoa_Acionista_Relacionado',
       'CPF_CNPJ_Acionista_Relacionado',
       'Quantidade_Acao_Ordinaria_Circulacao',
       'Percentual_Acao_Ordinaria_Circulacao',
       'Quantidade_Acao_Preferencial_Circulacao',
       'Percentual_Acao_Preferencial_Circulacao',
       'Quantidade_Total_Acoes_Circulacao',
       'Percentual_Total_Acoes_Circulacao', ]]
# fre_cia_aberta_posicao_acionaria_2023.columns


In [ ]:
file = "fre_cia_aberta_relacao_familiar_2023"
fre_cia_aberta_relacao_familiar_2023 = read(file)
# Parentescos
fre_cia_aberta_relacao_familiar_2023

In [ ]:
file = "fre_cia_aberta_relacao_subordinacao_2023"
fre_cia_aberta_relacao_subordinacao_2023 = read(file)
# Subordnicação 
fre_cia_aberta_relacao_subordinacao_2023

In [ ]:
file = "fre_cia_aberta_transacao_parte_relacionada_2023"
fre_cia_aberta_transacao_parte_relacionada_2023 = read(file)
# Partes relacionadas
fre_cia_aberta_transacao_parte_relacionada_2023

In [ ]:
file = "fre_cia_aberta_2023"
fre_cia_aberta_2023 = read(file)
# Link do Documento
fre_cia_aberta_2023

In [ ]:
def plot_group(cias_por_setor, window):
    for company, group_df in cias_por_setor:
        try:
            # Calculate the moving average for the last 4 periods
            group_df['MA'] = group_df['VL_CONTA'].rolling(window=window).mean()
            
            # Calculate the rolling sum for the last 4 periods
            group_df['Rolling_Sum'] = group_df['VL_CONTA'].rolling(window=window).sum()
            
            # Calculate the lifelong cumulative sum
            group_df['Cumulative_Sum'] = group_df['VL_CONTA'].cumsum()
            
            # Plot raw data
            # group_df['VL_CONTA'].plot(label='Raw Data', legend=True)

            # Plot moving average
            group_df['MA'].plot(label=f'{window} Quarters Moving Average', legend=True, linestyle='--')
            
            # Plot rolling sum
            group_df['Rolling_Sum'].plot(label=f'{window} Quarters Sum', legend=True, linestyle='-.')
            
            # Plot lifelong cumulative sum
            group_df['Cumulative_Sum'].plot(label='Lifelong Cumulative Sum', legend=True, linestyle='-.')

            plt.title(f"{group_df['CD_CVM'].iloc[-1]} {group_df['DENOM_CIA'].iloc[-1]}")
            plt.show()
        except Exception as e:
            print(f"Error plotting for {company}: {e}")
    return True

cias_por_setor = df[(df['AGRUPAMENTO'] == 'con') & (df['CD_CONTA'] == '3.11')].groupby('CD_CVM')
window = 4*1
plot_group(cias_por_setor, window)

In [ ]:
import matplotlib.pyplot as plt

cias_por_setor = df[(df['AGRUPAMENTO'] == 'con') & (df['CD_CONTA'] == '2.03')].groupby('DENOM_CIA')

fig, ax = plt.subplots(figsize=(10, 6))

for company, group_df in cias_por_setor:
    try:
        group_df[['VL_CONTA']].plot(ax=ax, label=company)
    except:
        print(company)

ax.set_title("VL_CONTA by Company")
ax.set_ylabel("VL_CONTA")
ax.set_xlabel("Index")
ax.legend(loc="best")

plt.show()
